In [9]:
#创建集合单元
from pymilvus import FieldSchema, CollectionSchema, DataType, MilvusClient

client = MilvusClient(uri="http://192.168.200.130:19530")

def create_collection():
    # 定义字段 schemas
    id_field = FieldSchema(name="id", dtype=DataType.INT64, auto_id=True, is_primary=True)
    url_field = FieldSchema(name="url", dtype=DataType.VARCHAR, max_length=65535)
    content_field = FieldSchema(name="content", dtype=DataType.VARCHAR, max_length=65535)
    content_vector_field = FieldSchema(name="content_vector", dtype=DataType.FLOAT_VECTOR, dim=1024)
    title_field = FieldSchema(name="title", dtype=DataType.VARCHAR, max_length=65535)
    title_vector_field = FieldSchema(name="title_vector", dtype=DataType.FLOAT_VECTOR, dim=1024)
    publish_time_field = FieldSchema(name="publish_time", dtype=DataType.INT64)

    # 创建 Collection schema
    schema = CollectionSchema(
        fields=[id_field, url_field, content_field, content_vector_field, title_field, title_vector_field, publish_time_field],
        description="CUMT_GPT的bge数据集"
    )

    # 创建 Collection
    collection_name = "cumt_gpt_bge"
    if client.has_collection(collection_name):
        client.drop_collection(collection_name)

    client.create_collection(collection_name=collection_name, schema=schema)

    # 为 content_vector 创建索引
    # index_params = client.prepare_index_params()
    # index_params.add_index(
    #     field_name="content_vector",
    #     index_type="IVF_FLAT",
    #     metric_type="COSINE",
    #     params={"nlist": 128}
    # )
    
    # HNSW
    index_params = client.prepare_index_params()
    index_params.add_index(
        field_name="content_vector",
        index_type="HNSW",
        metric_type="COSINE",
        params={"M": 32, "efConstruction": 250}
    )
    index_params.add_index(
        field_name="title_vector",
        index_type="HNSW",
        metric_type="COSINE",
        params={"M": 32, "efConstruction": 250}
    )

    client.create_index(
        collection_name=collection_name,
        index_params=index_params,
        sync=False)


create_collection()
client.list_indexes(collection_name="cumt_gpt_bge")

['content_vector', 'title_vector']

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
# 插入数据单元
import json
import time
from datetime import datetime
from pymilvus import MilvusClient
from langchain_community.embeddings import OllamaEmbeddings
from FlagEmbedding import FlagModel
from sklearn.preprocessing import normalize
# from sonyflake import SonyFlake

client = MilvusClient(uri="http://192.168.200.130:19530")

def emb_text(texts):
    # 使用 bert-large-chinese 模型
    # embedding_model = SentenceTransformer('maidalun1020/bce-embedding-base_v1')
    
    embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
    embedding_vectors = embedding_model.embed_documents(texts)
    # embedding_vectors = normalize(embedding_vectors, norm='l2')
    
    return embedding_vectors

# 文档列表
# docs = [
#     "Artificial intelligence was founded as an academic discipline in 1956.",
#     "Alan Turing was the first person to conduct substantial research in AI.",
#     "Born in Maida Vale, London, Turing was raised in southern England.",
# ]
# doc = "矿小助是一款由FlyingStudio（中国矿业大学翔工作室）开发维护的校园软件。提供课表、考试、成绩、校车校历、图书馆藏、校卡余额、宿舍电量等查询功能；同时具有课表导入日历、加权成绩计算、校园网自动登录、个性化主题背景等实用功能。"

contents = []
titles = []

# 打开 JSON 文件
# filename = '../raw_data/articles.json'
# filename = ('../raw_data/kxz.json')
# filename = '../raw_data/news.json'
filename = '../raw_data/矿大新闻网.json'
with open(filename, 'r', encoding='utf-8') as f:
    python_data = json.load(f)
    for data in python_data['data']:
        contents.append(data.get("content"))
        titles.append(data.get("title"))

# 设置每批次处理的数据量
batch_size = 100  # 每批次插入1000条数据

total_length = len(contents)
for start in range(0, total_length, batch_size):
    end = min(start + batch_size, total_length)
    
    # 分批生成内容向量和标题向量
    content_batch = contents[start:end]
    title_batch = titles[start:end]
    
    content_vectors = emb_text(content_batch)
    title_vectors = emb_text(title_batch)
    
    # 要插入的数据
    datas = []
    for i, data in enumerate(python_data['data'][start:end]):
        # 使用 strptime 将字符串解析为日期对象
        date_obj = datetime.strptime(data.get("date"), "%Y-%m-%d")
        
        # 使用 strftime 格式化为所需的格式，并将其转换为整数
        date_int = int(date_obj.strftime("%Y%m%d"))

        datas.append({
                # "id": sonyflake_id,
                "url": data.get("url"),
                "content": data.get("content"),
                "content_vector": content_vectors[i],
                "title": data.get("title"),
                "title_vector": title_vectors[i],
                "publish_time": date_int
            })

    # 插入当前批次的数据
    res = client.insert(collection_name="cumt_gpt_bge", data=datas)
    print(f"已插入第 {start // batch_size + 1} 批数据，共插入 {len(datas)} 条记录")
    time.sleep(5)  # 等待5秒
   

# print(res)


已插入第 1 批数据，共插入 100 条记录
已插入第 2 批数据，共插入 100 条记录
已插入第 3 批数据，共插入 100 条记录
已插入第 4 批数据，共插入 100 条记录
已插入第 5 批数据，共插入 100 条记录
已插入第 6 批数据，共插入 100 条记录


In [1]:
from pymilvus import AnnSearchRequest, WeightedRanker, RRFRanker
# 测试向量搜索单元
# from langchain_community.embeddings import OllamaEmbeddings
# from sentence_transformers import SentenceTransformer
from pymilvus import FieldSchema, CollectionSchema, DataType, MilvusClient
from FlagEmbedding import FlagModel
from langchain_community.document_transformers import LongContextReorder
import numpy as np

client = MilvusClient(uri="http://192.168.200.130:19530")


def emb_text(texts):
    # embedding_model = SentenceTransformer('maidalun1020/bce-embedding-base_v1')
    # embedding_vectors = embedding_model.encode(text, batch_size=1024)
    embedding_model = FlagModel('chuxin-llm/Chuxin-Embedding', use_fp16=True)
    embedding_vectors = embedding_model.encode_queries(texts, batch_size=256)

    # embedding_vectors = normalize(embedding_vectors, norm='l2')

    return embedding_vectors


doc = ["孙杨什么时候来矿大"]

doc_vector = emb_text(doc)

search_params = {
    "metric_type": "COSINE",
    "params": {}
}

# IVF_FLAT
# res = client.search(
#     collection_name="cumt_gpt_qwen",
#     data=doc_vector,
#     limit=10,
#     output_fields=["id", "url", "content", "publish_time"],
#     search_params=search_params
# )

# HNSW
# res = client.search(
#   collection_name="cumt_gpt_chuxin", # Collection name
#   data=doc_vector, # Replace with your query vector
#   search_params={
#     "metric_type": "COSINE",
#     "params": {"ef":150, "radius":0.36}, # Search parameters
#   }, # Search parameters
#   limit=10, # Max. number of search results to return
#   output_fields=["id","url","content","publish_time"], # Fields to return in the search results
#   consistency_level="Bounded"
# )

res = client.hybrid_search(
    collection_name="cumt_gpt_bge",
    reqs=[
        AnnSearchRequest(
            data=doc_vector,  # 查询向量
            anns_field="content_vector",  # 搜索字段
            param={"metric_type": "COSINE", "params": {"ef": 150, "radius": 0.3}},  # 搜索参数
            limit=10,  # 返回前10个结果
            # expr="publish_time > 20240802"  # 过滤条件
        ),
        AnnSearchRequest(
            data=doc_vector,  # 查询向量
            anns_field="title_vector",  # 搜索字段
            param={"metric_type": "COSINE", "params": {"ef": 150, "radius": 0.3}},  # 搜索参数
            limit=10,  # 返回前10个结果
            # expr="publish_time > 20240802"  # 过滤条件
        )
    ],
    ranker=WeightedRanker(float(0.7), float(0.3)),
    # ranker=RRFRanker(),
    limit=5,
    output_fields=["id", "url", "content", "title", "publish_time"],
    timeout=60,
    consistency_level="Bounded"
    # partition_names: Optional[List[str]] = None
)

reordering = LongContextReorder()
reordered_docs = reordering.transform_documents(res[0])

print(res)
print("----------------")
print(reordered_docs)

E:\Python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



data: ["[{'id': 453372567099005759, 'distance': 0.5041909217834473, 'entity': {'publish_time': 20241011, 'id': 453372567099005759, 'url': 'https://www.cumt.edu.cn//info/1127/68141.htm', 'content': '【镜湖大讲堂】（2024-15）信念·挑战·超越——我与冠军面对面\\n日期：2024-10-11浏览量：\\n学术公告\\n\\n讲座主题：2024年镜湖大讲堂系列讲座之十五——信念·挑战·超越：我与冠军面对面（zsbgsp-202410-004）\\n分享人：孙杨奥运冠军\\n时间：2024年10月16日15：00\\n地点：中国矿业大学镜湖大讲堂\\n主办方：共青团中国矿业大学委员会\\n中国矿业大学镜湖讲座中心\\n\\n分享嘉宾介绍：\\n孙杨，中国男子游泳运动员，国际级运动健将，以其在长距离自由泳项目中的卓越表现而闻名于世。男子1500米自由泳世界纪录保持者男子400米自由泳奥运会纪录保持者，2012年伦敦奥运会男子400米自由泳、男子1500米自由泳冠军；2016年里约奥运会男子200米自由泳冠军；2024年全国夏季游泳锦标赛男子400米自由泳冠军。世界泳坛历史上唯一一位男子200米自由泳、男子400米自由泳、男子1500米自由泳的奥运会世锦赛大满贯冠军得主，史上唯一一位男子400米自由泳世锦赛四连冠，唯一一位男子800米自由泳世锦赛三连冠，男子自由泳个人单项金牌数居世界第一。\\n\\n内容简介：\\n劈波斩浪，铸就辉煌。中国的水上竞速传奇，始于无数清晨的第一缕阳光与最后一抹晚霞间的不懈锤炼。中国游泳运动，深受“坚韧不拔，以达彼岸”“技精源于心诚”等传统体育精神与奥林匹克理念的双重滋养，锻造了既注重个体突破极限、亦强调团队协作荣耀的“水陆兼修”训练体系，以及以“赛场内外皆君子”“胜在毫厘，贵在千里”等为核心的文化底蕴。本次活动，泳坛名将孙杨将以自身奋斗历程为线索，引领我们深入探索竞技体育的魅力，细致解读专业训练、心理调适、战略规划的关键所在，引领我们深入理解如何在全球化浪潮中，继续书写中国体育运动的新篇章。\\n\\n下一条：【镜湖大讲堂】（2024-14）—